In [1]:
# ================================================
# 🔧 SETUP - Add src to Python Path
# ================================================

import sys
import os

# Add src directory to Python path so 'core' module can be found
project_root = os.getcwd()
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✅ Added to Python path: {src_path}")
else:
    print(f"✅ Already in path: {src_path}")

# Verify
print(f"📂 Working directory: {project_root}")
print(f"🔍 Python will search for modules in: {src_path}")
print("=" * 50)

✅ Added to Python path: d:\Dev\trading-bot\src
📂 Working directory: d:\Dev\trading-bot
🔍 Python will search for modules in: d:\Dev\trading-bot\src


In [13]:
import sys
import os

# Add src directory to Python path so 'core' module can be found
project_root = os.getcwd()
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✅ Added to Python path: {src_path}")
else:
    print(f"✅ Already in path: {src_path}")

# Verify
print(f"📂 Working directory: {project_root}")
print(f"🔍 Python will search for modules in: {src_path}")
print("=" * 50)

# ================================================
# 🧪 TEST PROGRESSIVE TRAINING (100 candles only)
# ================================================

print("🧪 PROGRESSIVE TEST - Training on 100 candles")
print("=" * 70)

import os
import pandas as pd
from src.training.model_trainer import SimpleModelTrainer

data_folder = os.path.join(os.getcwd(), 'data')

# Training files
training_files = {
    'M': os.path.join(data_folder, 'BTCUSDT-M.json'),
    'W': os.path.join(data_folder, 'BTCUSDT-W.json'),
    'D': os.path.join(data_folder, 'BTCUSDT-D.json'),
    '1h': os.path.join(data_folder, 'BTCUSDT-1h.json'),
    '15m': os.path.join(data_folder, 'BTCUSDT-15m.json'),
    'parquet_path': os.path.join(data_folder, 'levels_cache', 'BTCUSDT-15m-levels.parquet.full'),
}

print(f"\n🔍 DEBUG: training_files keys: {list(training_files.keys())}")
print(f"🔍 DEBUG: 'parquet_path' in training_files: {'parquet_path' in training_files}")

# Initialize trainer
test_trainer = SimpleModelTrainer()

start_time = pd.Timestamp('2023-04-03 13:45:00')
end_time = pd.Timestamp('2025-10-05 00:00:00')
""" start_time = pd.Timestamp('2023-04-10 00:00:00')
end_time = pd.Timestamp('2023-05-15 23:59:59') """
# Configure training with time range filter
test_trainer.configure_training(
    profit_threshold=3.0,
    loss_threshold=-2.0,
    lookforward_periods=[5, 10, 20],
    start_time=start_time,
    end_time=end_time,
)

print(f"\n⏱️  Starting progressive training...")
print(f"   Time range: {start_time} to {end_time}")
print(f"   This will process each candle individually\n")

# Train the model
success = test_trainer.train_model(
    training_files=training_files,
    level_timeframes=['M', 'W', 'D', '1h', '15m']
)

if success:
    info = test_trainer.get_model_info()
    print(f"\n🎉 TEST TRAINING SUCCESS!")
    print(f"   Model Type: {info['model_type']}")
    print(f"   Accuracy: {info['accuracy']:.1%}")
    print(f"   Features: {info['features']}")
    print(f"   Classes: {info['classes']}")
    print(f"\n✅ Progressive approach is working correctly!")
    print(f"   You can now train on full dataset")
else:
    print("❌ Test training failed!")

✅ Already in path: d:\Dev\trading-bot\src
📂 Working directory: d:\Dev\trading-bot
🔍 Python will search for modules in: d:\Dev\trading-bot\src
🧪 PROGRESSIVE TEST - Training on 100 candles

🔍 DEBUG: training_files keys: ['M', 'W', 'D', '1h', '15m', 'parquet_path']
🔍 DEBUG: 'parquet_path' in training_files: True
✅ Level extractor initialized with timeframe caching
✅ Level extractor initialized with timeframe caching

⏱️  Starting progressive training...
   Time range: 2023-04-03 13:45:00 to 2025-10-05 00:00:00
   This will process each candle individually

🎓 TRAINING MODEL

📋 Validating training files...
   ✅ Main file (15m): d:\Dev\trading-bot\data\BTCUSDT-15m.json
   ✅ Parquet file: d:\Dev\trading-bot\data\levels_cache\BTCUSDT-15m-levels.parquet.full

📊 Validating level timeframes: ['M', 'W', 'D', '1h', '15m']

✅ All 6 level timeframe files validated!
   Level timeframes: ['M', 'W', 'D', '1h', '15m', 'parquet_path']

📦 Loading precomputed features from parquet...
   ✅ Loaded 87,891 cand

Level features: 100%|██████████| 87882/87882 [00:09<00:00, 9022.99it/s]


   ✅ Level features calculated in 10.03s (37 features)

💾 Step 4/4: Adding memory features...
   ✅ Memory features added in 0.000s (10 features)

🔗 Combining all feature groups...
   ✅ Features combined in 0.005s

✅ Features ready: 87882 samples, 58 features
   TA: 11, Level: 37, Memory: 10
🏷️  Generating trading labels...
🏷️  Generating labels based on price movements...
   BUY signals: 1215 (1.4%)
   SELL signals: 3291 (3.7%)
   HOLD signals: 83376 (94.9%)
✅ Training data ready: 87882 samples, 58 features
📊 Label distribution:
   hold: 83376 (94.9%)
   sell: 3291 (3.7%)
   buy: 1215 (1.4%)

✅ TRAINING DATA PREPARATION COMPLETE
⏱️  Total time: 26.79 seconds (0.45 minutes)
📊 Total samples: 87,882
📊 Total features: 58
⚡ Average time per sample: 0.0003 seconds

🔥 Training with 70305 samples, 58 features
🖥️  GPU Detection: ✅ Available
   🚀 Training with GPU...
     ✅ GPU Training Complete: 4.06s | Accuracy: 96.8%

✅ TRAINING COMPLETE!
   Model: XGBoost-GPU
   Test Accuracy: 96.8%

🔍 Top 1

In [16]:
# ================================================
# 🚀 VECTORBT BACKTESTING - Replacing Manual Simulation
# ================================================

print("🚀 VECTORBT BACKTESTING MODE")
print("=" * 55)

import os
from typing import Dict, cast
import pandas as pd
import json

from src.backtesting.vectorbt_engine import VectorBTBacktester
from src.core.trading_types import ChartInterval
from src.training.data_loader import DataLoader
from src.training.model_trainer import SimpleModelTrainer

# Load model from file (independent from training cell)
simulation_trainer = SimpleModelTrainer()

try:
    success = simulation_trainer.load_model()
    if not success:
        print("❌ Failed to load model from file")
        print("   Make sure you've trained the model first (run Cell 2)")
        simulation_trainer = None
except Exception as e:
    print(f"❌ Error loading model: {e}")
    import traceback
    traceback.print_exc()
    simulation_trainer = None

if simulation_trainer is not None and simulation_trainer.is_trained:
    print("✅ Model loaded successfully!")
    
    # Display model info
    info = simulation_trainer.get_model_info()
    print(f"   Model: {info['model_type']} | Accuracy: {info['accuracy']:.1%} | Features: {info['features']}")
    
    # Configuration
    test_symbol = 'BTCUSDT'
    MAX_CANDLES = 500  # REDUCED for progressive backtesting (each candle recalculates levels!)
    test_data_folder = 'data'
    
    # Threshold settings
    buy_threshold = 0.10
    sell_threshold = 0.10
    
    # VectorBT settings
    initial_cash = 1000000.0
    commission = 0.001  # 0.1%
    slippage = 0.0005   # 0.05%
    
    print(f"\n🎯 Symbol: {test_symbol}")
    print(f"📊 Backtesting: {MAX_CANDLES} candles (PROGRESSIVE - no data leakage)")
    print(f"⚙️  Thresholds: BUY≥{buy_threshold:.0%}, SELL≥{sell_threshold:.0%}")
    print(f"💰 Initial Cash: ${initial_cash:,.2f}")
    print(f"💸 Commission: {commission:.2%} | Slippage: {slippage:.2%}\n")
    
   
    
    # Get file paths
    test_files: Dict[ChartInterval, str] = {
        'M': os.path.join(test_data_folder, f'{test_symbol}-M.json'),
        'W': os.path.join(test_data_folder, f'{test_symbol}-W.json'),
        'D': os.path.join(test_data_folder, f'{test_symbol}-D.json'),
        '1h': os.path.join(test_data_folder, f'{test_symbol}-1h.json'),
        '15m': os.path.join(test_data_folder, f'{test_symbol}-15m.json')
    }
    

    # Load 15m data (primary trading timeframe)
    print("📊 Loading 15m data...")
    test_data = DataLoader.load_single_json_file(test_files['15m'])

    if test_data is None:
        raise ValueError("❌ Could not load test data")

    print(f"✅ Loaded {len(test_data)} total candles")
    print(f"📅 Range: {test_data.index.min()} to {test_data.index.max()}")
    
    # Get a RANDOM segment of MAX_CANDLES for backtesting
    if len(test_data) < MAX_CANDLES:
        print(f"⚠️  Only {len(test_data)} candles available, using all")
    else:
        import random
        # Calculate valid range for random start position
        max_start = len(test_data) - MAX_CANDLES
        random_start = random.randint(0, max_start)
        random_end = random_start + MAX_CANDLES
        
        print(f"🎲 Randomly selecting {MAX_CANDLES} candles from position {random_start} to {random_end}")
        test_data = test_data.iloc[random_start:random_end]
    
    print(f"\n🎬 Starting progressive backtest with {len(test_data)} candles...")
    print(f"📍 Backtest period: {test_data.index.min()} to {test_data.index.max()}")
    
    # Load all timeframe data as DataFrames (needed for level extraction)
    print("\n📂 Loading multi-timeframe data for level extraction...")
    data_dfs: Dict[ChartInterval, pd.DataFrame] = DataLoader.load_multiple_json_files(test_files)
    
    print(f"✅ Loaded {len(data_dfs)} additional timeframes for levels\n")
    
    print("\n" + "=" * 55)
    print("🚀 VECTORBT PROGRESSIVE BACKTEST STARTING...")
    print("=" * 55)
    print("⚠️  Note: Progressive mode recalculates levels for EACH candle")
    print("   This is SLOW but prevents data leakage (realistic trading)")
    print("   Progress bar will show each candle being processed\n")
    
    # Initialize VectorBT backtester
    backtester = VectorBTBacktester(
        trainer=simulation_trainer,
        initial_cash=initial_cash,
        commission=commission,
        slippage=slippage
    )
    
    # Generate signals progressively (includes level extraction per candle)
    signals_df = backtester.generate_signals(
        data=test_data,
        data_dfs=data_dfs,
        buy_threshold=buy_threshold,
        sell_threshold=sell_threshold,
        timeframe='15m'
    )
    
    # Run backtest (THIS IS FAST - vectorized)
    print("\n🚀 Running backtest...")
    portfolio = backtester.run_backtest(signals_df=signals_df, freq='15T')
    
    # Display performance summary
    backtester.print_performance_summary()
    
    # Get trade analysis
    trades_df = backtester.get_trade_analysis()
    
    print("\n" + "=" * 55)
    print("📋 TRADE ANALYSIS")
    print("=" * 55)
    
    if len(trades_df) > 0:
        print(f"\n📊 Total Trades: {len(trades_df)}")
        
        winning_trades = trades_df[trades_df['PnL'] > 0]
        losing_trades = trades_df[trades_df['PnL'] < 0]
        
        print(f"✅ Winning: {len(winning_trades)} ({len(winning_trades)/len(trades_df)*100:.1f}%)")
        print(f"❌ Losing:  {len(losing_trades)} ({len(losing_trades)/len(trades_df)*100:.1f}%)")
        
        if len(winning_trades) > 0:
            print(f"\n💚 Average Win: ${winning_trades['PnL'].mean():.2f}")
            print(f"💚 Largest Win: ${winning_trades['PnL'].max():.2f}")
        
        if len(losing_trades) > 0:
            print(f"💔 Average Loss: ${losing_trades['PnL'].mean():.2f}")
            print(f"💔 Largest Loss: ${losing_trades['PnL'].min():.2f}")
        
        # Show sample trades
        print(f"\n📋 Sample Trades (first 10):")
        print(trades_df.head(10))
    else:
        print("⚠️  No trades executed during backtest period")
    
    print("\n" + "=" * 55)
    print("✅ VECTORBT BACKTEST COMPLETE!")
    print("=" * 55)
    
    # Signal distribution
    signal_counts = signals_df['signal'].value_counts()
    print(f"\n🎯 SIGNAL DISTRIBUTION:")
    for signal in ['BUY', 'SELL', 'HOLD']:
        count = signal_counts.get(signal, 0)
        pct = (count / len(signals_df) * 100) if len(signals_df) > 0 else 0
        print(f"   {signal}: {count} ({pct:.1f}%)")
    
    # Show trading signals (non-HOLD)
    trade_signals = signals_df[signals_df['signal'] != 'HOLD']
    print(f"\n📋 TRADING SIGNALS ({len(trade_signals)} total):")
    if len(trade_signals) > 0:
        for timestamp, row in trade_signals.head(20).iterrows():
            emoji = "🟢" if row['signal'] == 'BUY' else "🔴"
            print(f"   {emoji} {timestamp} | " +
                    f"{row['signal']:4s} @ ${row['close']:8.2f} | " +
                    f"Conf: {row['confidence']:5.1%}")
    else:
        print("   No trading signals generated")
    
    print("\n✅ Progressive backtest complete!")
    print("   ℹ️  Each candle was processed independently with levels extracted")
    print("      at that point in time (no future data leakage)")
    
    # ================================================
    # 📊 OPTIONAL: GENERATE INTERACTIVE PLOTS
    # ================================================
    
    try:
        print("\n📊 Generating interactive plots...")
        # Use static plots (no anywidget dependency)
        backtester.plot_results(use_widgets=False)
    except Exception as e:
        print(f"⚠️  Could not generate plots: {e}")
        if "anywidget" in str(e):
            print("   ℹ️  anywidget not installed")
            print("   Solution: Restart the kernel (Kernel → Restart)")
        print("   Use VectorBT_Backtest.ipynb for full visualization")
    
else:
    print("❌ No model available")
    print("   Please run Cell 2 first to train the model")


🚀 VECTORBT BACKTESTING MODE
✅ Level extractor initialized with timeframe caching
✅ Level extractor initialized with timeframe caching
📁 Loading model from: src/models/simple_trading_model.joblib
✅ Model loaded: XGBoost-GPU (Accuracy: 96.8%)
✅ Model loaded successfully!
   Model: XGBoost-GPU | Accuracy: 96.8% | Features: 58

🎯 Symbol: BTCUSDT
📊 Backtesting: 500 candles (PROGRESSIVE - no data leakage)
⚙️  Thresholds: BUY≥10%, SELL≥10%
💰 Initial Cash: $1,000,000.00
💸 Commission: 0.10% | Slippage: 0.05%

📊 Loading 15m data...
✅ Loaded 101000 total candles
📅 Range: 2022-11-22 00:00:00 to 2025-10-09 03:00:00
🎲 Randomly selecting 500 candles from position 50333 to 50833

🎬 Starting progressive backtest with 500 candles...
📍 Backtest period: 2024-04-29 08:30:00 to 2024-05-04 13:15:00

📂 Loading multi-timeframe data for level extraction...

📂 PRE-LOADING ALL DATA FILES (one-time operation)...
   Loading M... ✅ 99 candles
   Loading W... ✅ 426 candles
   Loading D... ✅ 2,976 candles
   Loading 1

Extracting levels:   0%|          | 0/500 [00:00<?, ?it/s]

      🔧 zigzag_middleware: 0.011s
      🔧 volume_profile_middleware: 0.012s
   ⏱️  [W] Total: 0.024s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 24ms | Extract: 0ms | Cache: 0ms


Extracting levels:   0%|          | 1/500 [00:00<01:11,  6.93it/s]

      🔧 zigzag_middleware: 0.031s
      🔧 volume_profile_middleware: 0.051s
   ⏱️  [D] Total: 0.088s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 88ms | Extract: 0ms | Cache: 0ms
   ⏱️  [1h] Total: 0.013s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 13ms | Extract: 0ms | Cache: 0ms
   ⏱️  [1h] Total: 0.013s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 13ms | Extract: 0ms | Cache: 0ms
   ⏱️  [1h] Total: 0.014s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 14ms | Extract: 0ms | Cache: 0ms
   ⏱️  [1h] Total: 0.013s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 13ms | Extract: 0ms | Cache: 0ms
   ⏱️  [1h] Total: 0.014s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 14ms | Extract: 0ms | Cache: 0ms


...

   ⏱️  [1h] Total: 0.011s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 11ms | Extract: 0ms | Cache: 0ms
   ⏱️  [1h] Total: 0.011s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 11ms | Extract: 0ms | Cache: 0ms
   ⏱️  [1h] Total: 0.011s | Copy: 0ms | Filter: 0ms | Init: 0ms | Run: 11ms | Extract: 0ms | Cache: 0ms
   ✅ Levels extracted in 12.73s
🎯 Step 3/3: Calculating level features per candle...


Level features: 100%|██████████| 500/500 [00:00<00:00, 8065.69it/s]

   ✅ Level features calculated in 0.07s (37 features)
🔮 Generating predictions for 500 candles...


   ✅ Predictions generated in 0.17s

🔍 Signal Analysis:
   BUY signals: 0
   SELL signals: 6
⚠️  No BUY signals detected - adjusting strategy...
   Auto-adjusted BUY threshold: 0.0117
   Adjusted BUY signals: 50
   Adjusted SELL signals: 6

✅ OPTIMIZED BACKTESTING COMPLETE
⏱️  Total time: 12.97 seconds (0.22 minutes)
📊 Processed: 500 candles
   BUY: 50
   SELL: 2
   HOLD: 448
⚡ Average time per candle: 25.94 milliseconds


🚀 Running backtest...

🚀 Running VectorBT backtest...
   Initial Capital: $1,000,000.00
   Commission: 0.10%
   Slippage: 0.05%
✅ Backtest complete!

📊 VECTORBT BACKTEST PERFORMANCE SUMMARY

💰 Returns:
   Total Return:            6.03%
   Annualized Return:       0.00%
   Final Value:       $1,060,280.48

📈 Risk Metrics:
   Sharpe Ratio:            7.34
   Max Drawdown:            6.49%

🎯 Trading Performance:
   Total Trades:               2
   Win Rate:                0.00%
   Profit Factor:           0.00


📋 TRADE ANALYSIS

📊 Total Trades: 2
✅ Winning: 1 (50.0%)


✅ Plots displayed!
